In [ ]:
# python3.6

import random
import sys
from IPython.display import clear_output, display
from paho.mqtt import client as mqtt_client
import os
import time
from datetime import datetime
from jetbot import Robot

robot = Robot()

broker = '10.102.17.81'
port = 1884
topic = "python/mqtt"
# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 100)}'
# username = 'emqx'
# password = 'public'
signs_detected = 0
signs_obeyed = 0
STATE = 0

def connect_mqtt() -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        global STATE
        if STATE == 0:
            received = msg.payload.decode()
            #clear_output(wait=True)
            print("MQTT Broker messages:")
            try:
                #x, y, k = 0
                x = 0.0
                y = 0.0
                k = 0.0
                sign = -1
                mqtt_input = received.split(" ")
                instruction = mqtt_input[0]
                params = mqtt_input[1]

                if instruction == "Vision":
                    commands = params.split(",")
                    x = float(commands[0])
                    y = float(commands[1])
                    k = float(commands[2])
                    print(f"x: {x}, y: {y}, k: {k}")

                elif instruction == "DeepStream":
                    sign = int(params)
    #                 print(f"Roadsign: {sign}")
    #                 if sign > 0 and (signs_detected == signs_obeyed):
    #                     print(f"SIGN DETECTED")
    #                     signs_detected += 1
    #                     last = datetime.now()
    #                     while (datetime.now() - last).total_seconds() < 3.0:
    #                             robot.left_motor.value = max(min(0.34 + k, 1.0), 0.0)
    #                             robot.right_motor.value = max(min(0.34 - k, 1.0), 0.0)
    #                         print("CLOSING ON IT!")
    #                         #continue
    #                     print("STOPPING!")
    #                     robot.left_motor.value = 0.0
    #                     robot.right_motor.value = 0.0
    #                     while (datetime.now() - last).total_seconds() < 5.0:
    #                         print("STOP!")
    #                         #continue
    #                 if sign == 0 and signs_detected > signs_obeyed:
    #                     print("BYE OLD SIGN... WAITING FOR A NEW ONE")
    #                     signs_obeyed = signs_detected



                #os.system('clear')
                #print(received+'\n')
                 #  print(f"x: {x}, y: {y}, k: {k}")
                  #      sys.stdout.write("STOP")

                if sign > 0:
                    robot.left_motor.value = 0.0
                    robot.right_motor.value = 0.0
                    STATE = 1
                else:
                    robot.left_motor.value = max(min(0.34 + k, 1.0), 0.0)
                    robot.right_motor.value = max(min(0.34 - k, 1.0), 0.0)
                print(f"Traffic signs detected = {signs_detected} \t Traffic signs obeyed = {signs_obeyed}")

            except Exception as e:
                print(f"Error trying to find commands")
        elif STATE == 1:
            print("DEBUGGING")
    client.subscribe(topic)
    client.on_message = on_message


def run():
    client = connect_mqtt()
    subscribe(client)
    client.loop_forever()


if __name__ == '__main__':
    run()

In [2]:
from jetbot import Robot

robot = Robot()

In [33]:
robot.stop()

In [3]:
from datetime import datetime
from IPython.display import clear_output, display
last = datetime.now()
while (datetime.now() - last).total_seconds() < 2.0:
    clear_output(wait=True)
#    time.sleep(1)
    display(datetime.now())

datetime.datetime(2022, 10, 28, 17, 52, 7, 346907)